In [1]:
!pip install pytube
!pip install pydub
!pip install fastaudio
!pip install -U fastai

!pip install voila
!jupyter serverextension enable --sys-prefix voila

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fastaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.3/193.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fastai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of fastaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.1/190.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Enabling: voila
- Writing config: /usr/etc/jupyter
    - Validating...
      voila 0.5.5 OK


In [1]:
from pytube import YouTube
from moviepy import editor as mp
from pydub import AudioSegment
import wave, os
from fastai.vision.widgets import *
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *

  torchaudio.set_audio_backend(backend)



In [2]:
# Define parameters
from google.colab import drive
drive.mount('/content/drive')

LENGTH_OF_CLIP = 5000
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/Production Jazz vs Classical Classifier"
if BASE_PATH[-1] != '/': BASE_PATH += '/'

def _get_labels(x): return 1

Mounted at /content/drive


In [3]:
def download_and_split(path, link):
    title = download(link, path)
    split_mp4_into_wavs(path + "test")
    return title

In [4]:
def split_mp4_into_wavs(path, length=LENGTH_OF_CLIP):
    filename = path.split('/')[-1]
    # Insert Local Video File Path
    clip = mp.VideoFileClip(path + ".mp4")

    # Insert Local Audio File Path

    clip.audio.write_audiofile("output.wav",codec='pcm_s16le')
    clip.close()

    sound = AudioSegment.from_file("output.wav", format="wav", channels=1)

    j = 0
    while (j + 5000 < len(sound)):
        j += 5000
        chunk = sound[j:(j + 5000)]
        with open(path+"-%s.wav" % j, "wb") as f:
            chunk.export(f, format="wav")
    try:
        os.remove(path + ".mp4")
    except:
        print(filename + ".mp4 could not be removed")

In [5]:
def download(link, path):
    audio_stream = YouTube(link).streams.filter(file_extension="mp4")
    audio_stream.first().download(output_path=path, filename="test.mp4")
    return audio_stream.first().title

In [6]:
def predict_genre(onclick):
    folder = BASE_PATH + "test/"
    if not os.path.exists(folder):
      os.makedirs(folder)
    else:
      for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    link = yt_input.value
    video_title = download_and_split(BASE_PATH + "test/", link)
    files = os.listdir(BASE_PATH + "test/")

    learn = load_learner(BASE_PATH + "production_learner.pkl")

    results = 0
    for file in files:
      temp_path = BASE_PATH + "test/" + file
      pred = int(learn.predict(Path(temp_path))[0])
      #print(pred)
      results += pred
    percentage = 100 - (results * 100 // len(files)) if results < len(files) // 2 else results * 100 // len(files)
    output = "For " + video_title + "..."
    output += "Jazz " if results > len(files) // 2 else "Classical "
    output += "(Confidence: " + str(percentage) + "%)"
    output_text.value = output

In [7]:
# Define labels
subtitle, yt_text, output_text = widgets.Label(), widgets.Label(), widgets.Label()

# Title
subtitle.value = "Using a XResNet18 architecture and fastai's learning rate optimization (Smith, 2017), predict an audio clip's genre"

# YouTube entry
yt_text.value = "Link some YouTube audio"
yt_input = widgets.Text()

submission_btn = widgets.Button(description = 'Classify')
submission_btn.on_click(predict_genre)

VBox([widgets.Label("Music classifier:"), subtitle, yt_text, yt_input, submission_btn, output_text])

# Link: https://www.youtube.com/watch?v=0geT6FWeX8w